In [1]:
!pip install pandas openpyxl
!pip install holidays tensorflow xlrd
import pandas as pd
import numpy as np
import holidays
import os
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from datetime import datetime, timedelta
from tensorflow.keras.layers import Input

df_list = []

for year in range(2004, 2025):

  base_file = f'Native_Load_{year}'
  file_xlsx = base_file + '.xlsx'
  file_xls = base_file + '.xls'

  if os.path.exists(file_xlsx):
      df_year = pd.read_excel(file_xlsx, engine='openpyxl')
      file_name = file_xlsx
  elif os.path.exists(file_xls):
      df_year = pd.read_excel(file_xls)
      file_name = file_xls
  else:
        print(f"file for the year {year} not found.")
        continue

  df_year.rename(columns={df_year.columns[0]: 'timestamp'}, inplace=True)
  df_year = df_year[['timestamp', 'SCENT']].copy() # keeps only south central region + timestamp
  df_year.rename(columns={'SCENT': 'load'}, inplace=True) # rename SCENT --> load

  def fix_24_hour(ts):
      if '24:00' in ts:
          date_part = ts.split()[0]
          fixed_ts = pd.to_datetime(date_part) + pd.Timedelta(days=1)
          return fixed_ts.replace(hour=0, minute=0)
      return pd.to_datetime(ts)
  df_year['timestamp'] = df_year['timestamp'].astype(str).apply(fix_24_hour)
  df_year['timestamp'] = df_year['timestamp'].dt.round('h')

  # create the hour, day of week, month, and is_weekend
  df_year['hour'] = df_year['timestamp'].dt.hour
  df_year['dayofweek'] = df_year['timestamp'].dt.dayofweek
  df_year['day'] = df_year['timestamp'].dt.day
  df_year['year'] = df_year['timestamp'].dt.year
  df_year['month'] = df_year['timestamp'].dt.month
  df_year['is_weekend'] = df_year['dayofweek'] >= 5

  # check for holidays
  us_holidays = holidays.US(years=year)
  df_year['is_holiday'] = df_year['timestamp'].dt.date.isin(us_holidays)
  # drop daylight savings dupes
  df_year = df_year[~df_year['timestamp'].duplicated()]

  df_list.append(df_year)

df = pd.concat(df_list, ignore_index=True)
print("Combined shape:", df.shape)

temp_list = []

for file in ["austin, texas 2004-01-01 to 2006-12-31.xlsx", "austin, texas 2007-01-01 to 2009-12-31.xlsx", "austin, texas 2010-01-01 to 2012-12-31.xlsx", "austin, texas 2013-01-01 to 2016-12-31.xlsx", "austin, texas 2017-01-01 to 2020-12-31.xlsx", "austin, texas 2021-01-01 to 2024-12-31.xlsx"]:
    temp_df = pd.read_excel(file)
    temp_df.rename(columns={temp_df.columns[0]: 'timestamp'}, inplace=True)
    temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp']).dt.round('h')
    temp_df.rename(columns={temp_df.columns[1]: 'temperature'}, inplace=True)
    temp_list.append(temp_df)

df_temp = pd.concat(temp_list, ignore_index=True)
df = pd.merge(df, df_temp, on='timestamp', how='left')
df['temperature'] = df['temperature'].ffill()

# check num of rows
# df.head(30)

Combined shape: (184082, 9)


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

,timestamp,load,hour,dayofweek,day,year,month,is_weekend,is_holiday,temperature
0,2004-01-01 01:00:00,3639.115900,1,3,1,2004,1,False,True,62.8
1,2004-01-01 02:00:00,3495.158058,2,3,1,2004,1,False,True,62.6
2,2004-01-01 03:00:00,3322.697830,3,3,1,2004,1,False,True,62.8
3,2004-01-01 04:00:00,3201.716842,4,3,1,2004,1,False,True,62.8
4,2004-01-01 05:00:00,3163.737777,5,3,1,2004,1,False,True,62.8
5,2004-01-01 06:00:00,3214.383975,6,3,1,2004,1,False,True,62.7
6,2004-01-01 07:00:00,3286.231799,7,3,1,2004,1,False,True,64.1
7,2004-01-01 08:00:00,3296.791662,8,3,1,2004,1,False,True,64.9
8,2004-01-01 09:00:00,3393.138647,9,3,1,2004,1,False,True,66.4
9,2004-01-01 10:00:00,3639.074915,10,3,1,2004,1,False,True,66.4


In [2]:
# remove rows with NaNs after lagging
df = df.dropna()

#'temperature',
features = ['hour', 'day', 'dayofweek', 'month','is_weekend', 'is_holiday', 'year', 'temperature']

scaler_X = StandardScaler()
X = df[features]
y = df['load']
X = X[~y.isna()]
y = y.dropna()

In [3]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Any NaN in X_train? ", pd.isna(X_train).any())
print("Any infinite in X_train? ", (X_train == float('inf')).any())
print("Any NaN in y_train? ", y_train.isna().any())
print("Any infinite in y_train? ", (y_train == float('inf')).any())
print(f"Total NaNs in y: {y.isna().sum()} out of {len(y)} samples")

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # track validation loss
    factor=0.5,          # reduce LR by 50%
    patience=2,          # wait 5 epochs without improvement
    min_lr=1e-6          # lower bound
)

opt = Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss='mse', metrics=['mae'])

model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, callbacks=[reduce_lr], verbose=1)

# MAPE
y_pred = model.predict(X_test).flatten()
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print(f"MAPE on test set: {mape:.2f}%")

Any NaN in X_train?  False
Any infinite in X_train?  False
Any NaN in y_train?  False
Any infinite in y_train?  False
Total NaNs in y: 0 out of 184102 samples
Epoch 1/30
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 4040736.7500 - mae: 1052.1895 - val_loss: 298569.0000 - val_mae: 414.9472 - learning_rate: 0.0010
Epoch 2/30
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 258079.8281 - mae: 386.6838 - val_loss: 231025.2500 - val_mae: 363.1881 - learning_rate: 0.0010
Epoch 3/30
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 203072.6875 - mae: 340.3517 - val_loss: 186124.3281 - val_mae: 325.6112 - learning_rate: 0.0010
Epoch 4/30
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 188837.5469 - mae: 326.4491 - val_loss: 177381.3281 - val_mae: 315.0208 - learning_rate: 0.0010
Epoch 5/30
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 177189.8125 - mae: 314.9693 - val_loss: 195895.5938 - val_mae: 333.7910 - learning_rate: 0.0010
Epoch 6/30
4143/4143 ━━━━━━━━━━━━━━━━━━━━ 11

In [4]:
tomorrow = (pd.Timestamp.now(tz='US/Pacific').normalize().tz_convert(None)+ pd.Timedelta(days=1))
yesterday = tomorrow - pd.Timedelta(days=1)
print(f"Tomorrow's date: {tomorrow.date()}")
tomorrow_month = tomorrow.month
tomorrow_day = tomorrow.day

tomorrow_rows = []
for hour in range(24):
    temp_rows = df[(df['month'] == tomorrow_month) & (df['day'] == tomorrow_day) & (df['hour'] == hour)]
    temp_tomorrow = temp_rows['temperature'].mean() if not temp_rows.empty else None

    dayofweek = tomorrow.dayofweek
    day = tomorrow_day
    month = tomorrow_month
    is_weekend = int(dayofweek >= 5)
    is_holiday = int(tomorrow.date() in us_holidays)

    tomorrow_rows.append([
        hour,
        day, dayofweek, month,
        is_weekend, is_holiday,
        tomorrow.year,
        temp_tomorrow
    ])

X_tomorrow = pd.DataFrame(tomorrow_rows, columns=features)
X_tomorrow_scaled = scaler.transform(X_tomorrow)

predicted_loads = model.predict(X_tomorrow_scaled).flatten()

for hour, load in enumerate(predicted_loads):
    print(f"Hour {hour:02d}: {load:.2f} MW")

# print(tomorrow_df)
# df[(df['year'] == 2024) & (df['month'] == 8) & (df['day'] == 16)].dropna()

Tomorrow's date: 2025-08-24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
Hour 00: 10444.75 MW
Hour 01: 9773.38 MW
Hour 02: 9258.14 MW
Hour 03: 8850.04 MW
Hour 04: 8554.83 MW
Hour 05: 8272.88 MW
Hour 06: 8058.69 MW
Hour 07: 7931.81 MW
Hour 08: 8168.57 MW
Hour 09: 8662.56 MW
Hour 10: 9195.15 MW
Hour 11: 10037.18 MW
Hour 12: 10914.77 MW
Hour 13: 11719.43 MW
Hour 14: 12330.95 MW
Hour 15: 12865.42 MW
Hour 16: 13242.00 MW
Hour 17: 13507.86 MW
Hour 18: 13479.83 MW
Hour 19: 13367.87 MW
Hour 20: 12968.92 MW
Hour 21: 12638.80 MW
Hour 22: 12041.04 MW
Hour 23: 11288.62 MW
